In [1]:
import os, sys
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

# redirects "Using TensorFlow backend." printed by keras
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

In [3]:
#blockPrint()

import io
import numpy as np
path = os.getcwd()
# os.chdir(path)
from utils.parent_classes import Image
from PIL import Image as PILImage
import cv2

# MRCNN model
from model.mask_rcnn.config import Config
from model.mask_rcnn.loaders import MRCNNPriceDigitsModel, MRCNNPriceModel
from IPython.display import display, Audio
from gtts import gTTS
from ipywidgets import widgets, interact
from straighten_img import straightening_2, straightening


# needed parts of original pipeline
import pipeline.image_preparation as ip
import pipeline.price_reading as pr
#print(tf.__version__)

#tesseract
from tesserocr import PyTessBaseAPI

# reinitiates stderr
sys.stderr = stderr

In [4]:
#enablePrint()
# =============================================================================
# INITIALIZE MODELS
# =============================================================================
Config.RCNN_MODEL_PATH = os.path.join(*["..", "model", "weights"])

# build MRCNN digits model, CPU as default
mrcnn_price_digits_model = MRCNNPriceDigitsModel(DEVICE=Config.DEVICE)

# load pricetag detection model
mrcnn_price_model = MRCNNPriceModel(DEVICE=Config.DEVICE, AML_model_name="price")

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-core 1.1.5.3 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.11.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.1.5.3 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.11.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.1.5.3 (/anaconda/envs/azureml_py36/lib/python3.6/site-packages), Requirement.parse('azureml-core~=1.11.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with ex

In [5]:
# # vizualize detected pricetags
# image = pr.VisualizePriceTags.make_step(image, mrcnn_price_model)
# display(PILImage.fromarray(image.price_tags_detections_image))

In [110]:
def callback(value):
    if btn_upload.data:
        # =============================================================================
        # INITIALIZE IMAGE
        # =============================================================================

        #print("You upload a file:")
        im_file = PILImage.open(io.BytesIO(btn_upload.data[-1]))

        # initialize Image object
        photo = Image(image=im_file)

        # correct exif rotation of image
        photo = ip.ExifRotate.make_step(photo)
        
        # display original photo, resize it for faster display
        basewidth = 800
        wpercent = (basewidth/float(photo.image.size[0]))
        hsize = int((float(photo.image.size[1])*float(wpercent)))
        shelf_photo = photo.image.resize((basewidth,hsize), PILImage.ANTIALIAS)
#         display(shelf_photo)
        
        # =============================================================================
        # DETECT PRICETAGS
        # =============================================================================

        photo = pr.DetectPriceTags.make_step(photo, mrcnn_price_model)
        print("Nalezeno", len(photo.price_tags_detections["rois"]),"cenovek,\n čtu cenovku nejblíže středu snímku.\n")

        # =============================================================================
        # GET PRICE
        # =============================================================================

        ### select n pricetags closest to the center

        # get center of image
        photo.image = np.array(photo.image)
        img_center = np.array(photo.image.shape[0:2])/2

        # calculate distances from center of pricetags to center of image
        distances = [
            np.linalg.norm(
                np.array(
                    [
                        np.mean(r[[0,2]]),
                        np.mean(r[[1,3]])
                    ]
                )-img_center
            )
            for r in photo.price_tags_detections["rois"]
        ]

        # select n pricetags closest to the center
        # for now, n=1
        pricetags_indices = np.argsort(distances)[0:1]

        for ind_pricetag in pricetags_indices:
            # cut pricetag from image
            bbox = photo.price_tags_detections["rois"][ind_pricetag]
            img_size = photo.image.shape

            #use 10% wider bbox
            ratio = 0.05
            bbox_new = bbox
            bbox_new[0] = max(0,bbox[0]-ratio*(bbox[2]-bbox[0]))
            bbox_new[1] = max(0,bbox[1]-ratio*(bbox[3]-bbox[1]))
            bbox_new[2] = min(img_size[0],bbox[2]+ratio*(bbox[2]-bbox[0]))
            bbox_new[3] = min(img_size[1],bbox[3]+ratio*(bbox[3]-bbox[1]))
            pricetag = photo.image[
                bbox_new[0] : bbox_new[2], 
                bbox_new[1] : bbox_new[3],
                :
            ]

            #calculate rotation
            mask = photo.price_tags_detections["masks"][bbox_new[0]:bbox_new[2],bbox_new[1]:bbox_new[3],ind_pricetag]
            mask_bw = (mask*255).astype(np.uint8)

            #straightening
            src_points, dst_points = straightening_2(mask_bw)
            pricetag_rotated, angle = straightening(pricetag,src_points,dst_points)

            # convert to array and detect digits
            photo.digits_detections = pr.ReadPrices.detect_objects(
                Image(image=pricetag_rotated), 
                mrcnn_price_digits_model
            )

            #resize
            pricetag_rotated = cv2.resize(pricetag_rotated, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)

            # get final price from digits detections
            # file pipeline/price_reading.py, class ReadPrices, fcn read_price_from_detections
            photo.price = pr.ReadPrices.read_price_from_detections(
                photo.digits_detections,
                decimal_point=True
                )

            #header selection - crop on highest pixel of the highest digit
            digits_masks = photo.digits_detections['masks']
            if len(digits_masks) > 0:
                digits_masks = np.transpose(digits_masks,[2,0,1])
                max_y_size = 0
                for digit_mask in digits_masks:
                    y_mask_indices = np.where(digit_mask == np.amax(digit_mask))[0]
                    min_index = y_mask_indices[0]
                    max_index = y_mask_indices[-1]
                    y_size = max_index-min_index
                    if max_y_size < y_size:
                        max_y_size = y_size
                        overall_top_index = min_index
                ratio = 1.1
                overall_top_index *= ratio
                pricetag_rotated = pricetag_rotated[:int(overall_top_index*2.5),:]
            
            #grayscaling
            pricetag_rotated_gray = cv2.cvtColor(pricetag_rotated, cv2.COLOR_RGB2GRAY)

            #binarization by thresholding
            _,pricetag_rotated_gray_thresholded = cv2.threshold(pricetag_rotated_gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            
            #tesserocr
            with PyTessBaseAPI(lang='ces') as api:
                pricetag_rotated_gray_thresholded_pil = PILImage.fromarray(pricetag_rotated_gray_thresholded)
                api.SetImage(pricetag_rotated_gray_thresholded_pil)
                nazev = api.GetUTF8Text()
                if len(nazev.strip())>0:
                    print('Název výrobku je:\n',nazev.strip())
                else:
                    print('Nedokážu přečíst název výrobku, zkuste prosím přijít blíže k regálu.\n')
                 

            print('\Cena je:\n ',photo.price, 'korun.')
            #display(pricetag_rotated_gray_thresholded)
            display(PILImage.fromarray(pricetag))
    return

In [111]:
btn_upload = widgets.FileUpload()

In [112]:
_ = interact(callback, value=btn_upload)

interactive(children=(FileUpload(value={}, description='Upload'), Output()), _dom_classes=('widget-interact',)…